# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [ ]:
# Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [ ]:
# Open the inspect element feature in your browser

In [29]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
import re
import time

In [7]:
html_page = requests.get("https://www.residentadvisor.net/events")
soup = BeautifulSoup(html_page.content, 'html.parser')

In [10]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en,ja,es">
<head id="_x1"><title>
	RA: Events in Georgia, United States of America
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en,ja,es" http-equiv="content-language"/><meta content="RA: Resident Advisor" name="Description"/><meta content="RA, residentadvisor, resident, advisor, music, ra, events, in, georgia, united, states, america" name="Keywords"/><meta content="Resident Advisor" name="Author"/><meta content="Resident Advisor" property="og:site_name"/><meta content="712773712080127" property="fb:app_id"/><link href="/bundles/default-css?v=FkfRVAlFvpndxqgZliJaJOXD-OhkiRFP8nrBK9Pg2R01" rel="stylesheet"/>
<meta content="app-id=981952703, app-argument=ra-guide://search" name="apple-itunes-app"/><link href="/bundles/cat-listings-css?v=qgpSmyPbylOKeJFqy2yvCrTgAsw9yQYcJtLKS_vPO6s1" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/vnd.microsoft.icon"/><link color="#0

<div class="pnlLeaderWrapper">
<div id="pnlLeader">
<aside class="div-gpt-ad-1452773484957-6 leader"><div class="ad" id="div-gpt-ad-1452773484957-6"></div></aside>
</div>
</div>

In [20]:
event_listings = soup.find('div', id="event-listing")

In [21]:
entries = event_listings.findAll('li')
print(len(entries), entries[0])

8 <li><p class="eventDate date"><a href="/events.aspx?ai=64&amp;v=day&amp;mn=1&amp;yr=2020&amp;dy=11"><span>Sat, 11 Jan 2020 /</span></a></p></li>


## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [24]:
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,Chad Andrew and Tocayo,The Alley Cat Music Club,"Sat, 11 Jan 2020 /",1.0
1,Atloc: B2B Massive - Iris Esp101 Learn to Beli...,Believe Music Hall,"Sat, 11 Jan 2020 /",NaN
2,G Jones - Iris Esp101 Learn to Believe,Believe Music Hall,"Thu, 16 Jan 2020 /",NaN
3,Markus Schulz,Ravine,"Fri, 17 Jan 2020 /",3.0
4,Shiverz - Shut Down and Shell Tour - Wish Lounge,Believe Music Hall,"Fri, 17 Jan 2020 /",NaN


In [25]:
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [26]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/georgia/week/2020-01-18'

In [27]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    #Your code here
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [ ]:
#Your code here
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!